# Task 2
Write a web scraper to collect the following real-time data for 20-30 min continuously with a
refresh frequency of 30 sec. Use the column “%chg” of real-time collected data (open the link to
look at the table) to alert when the difference between the %chg at two fixed time instances (2
minute or so) of the respective company crosses 2 percent. Writing this alert as a separate
function is preferable. Import this library and implement any of the following technical
indicators. Create seperate table for indicators.
** Table in question 2 will be updated real time only through Mon-Fri, 9:15 - 15:30, i.e. market
hours. Many of you have this doubt why the real time table is static.
** You may want to use NumPy of size [time x row x column], so that it can facilitate subtraction
between values of two fixed time instances.
** If this suggestion helps to implement Stockstats indicator: Use Last price at the beginning as
Open, and Close at the end of each time instance. You can implement your way though.

In [73]:
# Importing Dependencies 

#Note Scraper.py must be in the same dir as this notebook
from Scraper import Scraper
import numpy as np
import pandas as pd
import time
import datetime


##### Set Run-Time and Frequency for data-collection

In [74]:

run_time=20 #in mins
freq=60     #in seconds


In [75]:
# Initialize and use the scrapper object to download the data
scrp=Scraper()
df=pd.read_csv(scrp.get(),names=["Company Name","Industry","LastPrice","Change","%Chg","Mkt Cap(Rs cr)","Time"])


### Periodic Data Collections 

In [ ]:
history=pd.DataFrame()
for i in range(int(run_time*(60/freq))):
    try:
        prev_df=df
        df=pd.read_csv(scrp.get(),header=None,names=["Company Name","Industry","LastPrice","Change","%Chg","Mkt Cap(Rs cr)","Time"])
        df['Time']=datetime.datetime.now()
        history=history.append(df)
        delta=df[abs(df['%Chg'])-abs(prev_df['%Chg']) > 0.2]

        if(len(delta)>0):
            print("ALERT!!")
            for name,val in zip(delta["Company Name"].values,delta["%Chg"].values):
                print(name , " changed by ",val)
            print("***"*10)
        else:
            print("No change")
     
    except Exception as e:
        print(e)
        continue
    
    time.sleep(freq)
    
history.to_csv(f"{run_time}_min_history_period_1min.csv")

## Furthur Analysis

In [77]:
#The stored data is interpreted as below

history=pd.read_csv(f"{run_time}_min_history_period_1min.csv")
history.drop("Unnamed: 0",inplace=True,axis=1)
w=history.pivot_table(index=["Time","Company Name"])
w.sort_index(inplace=True)
w

%Chg  Change  LastPrice  \
Time                       Company Name                                
2020-03-24 15:02:36.074044 Adani Ports      10.64   22.10     229.90   
                           Asian Paints      0.80   12.05    1510.50   
                           Axis Bank        -0.96   -2.95     305.70   
                           BPCL             -4.55  -12.25     257.10   
                           Bajaj Auto       -0.62  -12.05    1923.50   
...                                           ...     ...        ...   
2020-03-24 15:21:53.731000 UltraTechCement  -1.20  -36.60    3018.25   
                           Vedanta           0.40    0.25      62.90   
                           Wipro             3.61    6.15     176.30   
                           Yes Bank        -12.33   -4.90      34.85   
                           Zee Entertain    -2.50   -3.05     118.95   

                                            Mkt Cap(Rs cr)  
Time                       Company Name                     
2020-03-24 15:02:36.074044 Adani Ports            46709.97  
                           Asian Paints          144886.83  
                           Axis Bank              86251.46  
                           BPCL                   55771.49  
                           Bajaj Auto             55659.75  
...                                                    ...  
2020-03-24 15:21:53.731000 UltraTechCement        87113.78  
                           Vedanta                23381.17  
                           Wipro                 100726.49  
                           Yes Bank               43738.40  
                           Zee Entertain          11424.95  

[1000 rows x 4 columns]

In [78]:
#Individual Company performance can be be sliced from the main dataframe as below
w.xs(level=1,key="Wipro").head()

,%Chg,Change,LastPrice,Mkt Cap(Rs cr)
Time,,,,
2020-03-24 15:02:36.074044,3.23,5.50,175.65,100355.12
2020-03-24 15:03:37.435325,3.61,6.15,176.30,100726.49
2020-03-24 15:04:38.816881,3.50,5.95,176.10,100612.22
2020-03-24 15:05:39.209829,3.56,6.05,176.20,100669.36
2020-03-24 15:06:41.097260,3.20,5.45,175.60,100326.56


In [79]:
#Individual time slices can also be derived from the same dataframe
w.xs(level=0,key=w.index[0][0]).head()

,%Chg,Change,LastPrice,Mkt Cap(Rs cr)
Company Name,,,,
Adani Ports,10.64,22.10,229.9,46709.97
Asian Paints,0.80,12.05,1510.5,144886.83
Axis Bank,-0.96,-2.95,305.7,86251.46
BPCL,-4.55,-12.25,257.1,55771.49
Bajaj Auto,-0.62,-12.05,1923.5,55659.75


### Slicing the MultiIndex dataframe to get periodic %Chg 

In [80]:
chg_per=w.unstack(level=0).loc[:,"%Chg"] # Unstack the time index and extract only the %Chg column
chg_per.head()

Time,2020-03-24 15:02:36.074044,2020-03-24 15:03:37.435325,2020-03-24 15:04:38.816881,2020-03-24 15:05:39.209829,2020-03-24 15:06:41.097260,2020-03-24 15:07:41.486425,2020-03-24 15:08:41.827910,2020-03-24 15:09:43.048637,2020-03-24 15:10:43.534815,2020-03-24 15:11:44.097434,2020-03-24 15:12:45.656326,2020-03-24 15:13:46.229051,2020-03-24 15:14:46.637657,2020-03-24 15:15:47.865486,2020-03-24 15:16:49.052486,2020-03-24 15:17:49.408034,2020-03-24 15:18:51.276519,2020-03-24 15:19:51.649835,2020-03-24 15:20:52.020249,2020-03-24 15:21:53.731000
Company Name,,,,,,,,,,,,,,,,,,,,
Adani Ports,10.64,11.04,10.90,10.85,10.85,10.85,10.92,10.88,10.88,11.04,11.04,11.16,11.16,11.26,11.04,11.04,11.67,11.62,11.62,12.51
Asian Paints,0.80,0.97,1.07,1.10,1.18,1.18,0.84,1.70,1.61,1.62,1.94,1.75,1.75,1.81,1.67,1.67,2.53,2.45,2.45,2.44
Axis Bank,-0.96,-1.81,-2.56,-2.22,-2.72,-2.72,-2.37,-1.72,-1.72,-1.99,-1.73,-1.47,-1.47,-1.57,-1.38,-1.38,-1.60,-1.65,-1.65,-2.15
BPCL,-4.55,-4.07,-4.21,-3.84,-4.01,-4.01,-4.29,-3.75,-3.73,-2.90,-1.82,-2.04,-2.04,-1.23,-1.61,-1.61,-1.69,-2.47,-2.47,-0.61
Bajaj Auto,-0.62,-0.59,-0.99,-1.02,-0.50,-0.50,-1.09,-0.64,-0.50,-0.72,0.57,0.55,0.55,0.55,0.41,0.41,0.27,0.44,0.44,0.67


In [81]:
# Dataframe to see %Chg surge/drop wrt a threshold over a period of time
thresh_hold=0.5
delta=chg_per.diff(periods=1,axis=1)>thresh_hold   # Calculating delta of %Chg for a given threshold
delta.head()

Time,2020-03-24 15:02:36.074044,2020-03-24 15:03:37.435325,2020-03-24 15:04:38.816881,2020-03-24 15:05:39.209829,2020-03-24 15:06:41.097260,2020-03-24 15:07:41.486425,2020-03-24 15:08:41.827910,2020-03-24 15:09:43.048637,2020-03-24 15:10:43.534815,2020-03-24 15:11:44.097434,2020-03-24 15:12:45.656326,2020-03-24 15:13:46.229051,2020-03-24 15:14:46.637657,2020-03-24 15:15:47.865486,2020-03-24 15:16:49.052486,2020-03-24 15:17:49.408034,2020-03-24 15:18:51.276519,2020-03-24 15:19:51.649835,2020-03-24 15:20:52.020249,2020-03-24 15:21:53.731000
Company Name,,,,,,,,,,,,,,,,,,,,
Adani Ports,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True
Asian Paints,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False
Axis Bank,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
BPCL,False,False,False,False,False,False,False,True,False,True,True,False,False,True,False,False,False,False,False,True
Bajaj Auto,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


In [82]:
#Dataframe indicating the vaule of delta of the %Chg coulmn 
chg_per.diff(periods=1,axis=1)[delta].head()

Time,2020-03-24 15:02:36.074044,2020-03-24 15:03:37.435325,2020-03-24 15:04:38.816881,2020-03-24 15:05:39.209829,2020-03-24 15:06:41.097260,2020-03-24 15:07:41.486425,2020-03-24 15:08:41.827910,2020-03-24 15:09:43.048637,2020-03-24 15:10:43.534815,2020-03-24 15:11:44.097434,2020-03-24 15:12:45.656326,2020-03-24 15:13:46.229051,2020-03-24 15:14:46.637657,2020-03-24 15:15:47.865486,2020-03-24 15:16:49.052486,2020-03-24 15:17:49.408034,2020-03-24 15:18:51.276519,2020-03-24 15:19:51.649835,2020-03-24 15:20:52.020249,2020-03-24 15:21:53.731000
Company Name,,,,,,,,,,,,,,,,,,,,
Adani Ports,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.63,NaN,NaN,0.89
Asian Paints,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.86,NaN,NaN,NaN
Axis Bank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BPCL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.54,NaN,0.83,1.08,NaN,NaN,0.81,NaN,NaN,NaN,NaN,NaN,1.86
Bajaj Auto,NaN,NaN,NaN,NaN,0.52,NaN,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Prepare data for format compatible with StockStats library

In [83]:
#calculate high and low for each stock over the period-"run_time"
ss_df=history[["Company Name","Time","LastPrice"]].pivot_table(index=["Company Name","Time"])
high=ss_df.max(level="Company Name")
low=ss_df.min(level="Company Name")


In [84]:
#calculate open and close for each stock at the start and end of the period-"run_time"
ss_df=history[["Company Name","Time","LastPrice"]].pivot_table(index=["Time","Company Name"])
ss_df.sort_index(level="Time",inplace=True)
open_df=ss_df.loc[ss_df.index[0][0]]   # first set of entry for the run_time period
close_df=ss_df.loc[ss_df.index[-1][0]] # last set of entry for the run_time period

In [85]:
#Concatenated all stats to form a dataframe 
ss_df=pd.concat([open_df,close_df,high,low],axis=1)

In [86]:
#renaming columns accordingly
ss_df.columns=["open","close","high","low"]

In [87]:
ss_df.head()

,open,close,high,low
Company Name,,,,
Adani Ports,229.9,233.80,233.80,229.90
Asian Paints,1510.5,1535.00,1536.35,1510.50
Axis Bank,305.7,302.00,305.70,300.25
BPCL,257.1,267.70,267.70,257.10
Bajaj Auto,1923.5,1948.45,1948.45,1914.40


In [88]:
import stockstats as ss

In [89]:
#Converting Pandas dataframe to stockstats dataframe
sdf=ss.StockDataFrame.retype(ss_df)

In [90]:
sdf.head()

,open,close,high,low
Company Name,,,,
Adani Ports,229.9,233.80,233.80,229.90
Asian Paints,1510.5,1535.00,1536.35,1510.50
Axis Bank,305.7,302.00,305.70,300.25
BPCL,257.1,267.70,267.70,257.10
Bajaj Auto,1923.5,1948.45,1948.45,1914.40
